In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model


In [2]:
train_path ="/kaggle/input/duckchick/train"
test_path ="/kaggle/input/duckchick/test"
val_path ="/kaggle/input/duckchick/val"



In [3]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)


In [4]:
x_test=[]

for folder in os.listdir(test_path):
    sub_path=test_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_test.append(img_arr)


In [5]:
x_val=[]

for folder in os.listdir(val_path):
    sub_path=val_path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_val.append(img_arr)


In [6]:
train_x=np.array(x_train)
test_x=np.array(x_test)
val_x=np.array(x_val)
train_x=train_x/255.0
test_x=test_x/255.0
val_x=val_x/255.0


In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')
test_set = test_datagen.flow_from_directory(test_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')
val_set = val_datagen.flow_from_directory(val_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'sparse')


Found 200 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [8]:
train_y=training_set.classes
test_y=test_set.classes
val_y=val_set.classes
print(training_set.class_indices)

{'chicken': 0, 'duck': 1}


In [9]:
vgg = VGG19(input_shape=(224,224,3), weights='imagenet', include_top=False)


80134624/80134624 [==============================] - 0s 0us/step


In [10]:
for layer in vgg.layers:
    layer.trainable = False


In [11]:
x = Flatten()(vgg.output)


In [12]:
prediction = Dense(2, activation='softmax')(x)


In [13]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [14]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)


In [15]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [16]:
history = model.fit(
  train_x,
  train_y,
  validation_data=(val_x,val_y),
  epochs=50,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)


Epoch 1/50
7/7 [==============================] - 77s 11s/step - loss: 1.1050 - accuracy: 0.5650 - val_loss: 0.5617 - val_accuracy: 0.7250
Epoch 2/50
7/7 [==============================] - 75s 11s/step - loss: 0.3874 - accuracy: 0.8250 - val_loss: 0.7568 - val_accuracy: 0.6750
Epoch 3/50
7/7 [==============================] - 74s 11s/step - loss: 0.1824 - accuracy: 0.9300 - val_loss: 0.7484 - val_accuracy: 0.6750
Epoch 4/50
7/7 [==============================] - 74s 11s/step - loss: 0.1551 - accuracy: 0.9350 - val_loss: 0.7319 - val_accuracy: 0.7000
Epoch 5/50
7/7 [==============================] - 75s 11s/step - loss: 0.1278 - accuracy: 0.9400 - val_loss: 0.5219 - val_accuracy: 0.7750
Epoch 6/50
7/7 [==============================] - 74s 11s/step - loss: 0.0502 - accuracy: 0.9950 - val_loss: 0.4792 - val_accuracy: 0.7750
Epoch 7/50
7/7 [==============================] - 74s 11s/step - loss: 0.0332 - accuracy: 0.9950 - val_loss: 0.4326 - val_accuracy: 0.7500
Epoch 8/50
7/7 [===========

In [17]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
y_pred=model.predict(test_x)
y_pred=np.argmax(y_pred,axis=1)
#get classification report
print(classification_report(y_pred,test_y))
#get confusion matrix
print(confusion_matrix(y_pred,test_y))


2/2 [==============================] - 13s 2s/step
              precision    recall  f1-score   support

           0       0.90      0.90      0.90        20
           1       0.90      0.90      0.90        20

    accuracy                           0.90        40
   macro avg       0.90      0.90      0.90        40
weighted avg       0.90      0.90      0.90        40

[[18  2]
 [ 2 18]]
